<a href="https://colab.research.google.com/github/SASambath69/notebooks/blob/main/Checkpoint_4_data_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contexte

Le présent notebook a été réalisé dans le cadre de ma formation de Data Analyst au sein de la Wild Code School de Lyon. Il 'agit d'un travail réalisé pour le 4ème et dernier Checkpoint (évaluation) de la formation. Pour ce Checkpoint prévu sur 48 heures, j'avais accès à 2 bases de données :

*   une base de données d'une société qui vend des albums musicaux (données fictives)
*   un dataset contenant une extraction de plus de 1 000 offres d'emploi dédiées aux métiers du numérique, publiées sur le site de Pôle Emploi.

Je devais choisir l'une des 2 bases et en proposer une analyse, comme si je travaillais dans une entreprise. J'ai choisi la 1ère sur les ventes d'albums musicaux.

# Objectifs Visés

En ce qui concerne l'orientation de mes travaux, plutôt que de proposer une analyse de la base de données, j'ai choisi de proposer des fonctionnalités en partant de la base. De cette manière, le fait que les données soient fictives n'allaient pas me perturber plus que cela.

### 1. Mise en place d'une solution de Reporting sur Excel et de Dashboarding sur Tableau, le tout automatisé

Ici, les cibles visées sont :
*    la direction en lui proposant un outil de pilotage en temps réel via la solution de Dashboarding
*    le service contrôle de gestion en lui fournissant un outil de Reporting automatisé

Cheminement :

1.   Création d'un script automatisé qui va récupérér les données des différentes tables
2.   Ce script va générer 2 fichiers
3.   1 fichier ".csv" qui sera automatiquement connecté au logiciel Tableau (déjà pré paramétré en amont) pour un dashboarding rapide
4.   1 fichier ".xlxs" qui contiendra différents tableaux de reporting

### 2. Mise en place d'un système de recommandations de musiques selon différents critères

Ici, la cible visée est :
*    les clients, en leur fournissant un système de recommandations de musiques en fonction de leurs préférences ou achats

Cheminement :

1.   Préparation des DataFrames qui serviront de base au système de recommandations (avec notamment des get_dummies)
2.   Mise en place de 3 systèmes de recommandations selon différents critères (Compositeur, Genre et Artiste/Album)

# Structure de la DataBase

![Structure](http://images.innoveduc.fr/data/SQL/SQLite_chinook.png)

# 1. Reporting et Dashboarding Automatisé

Une fois qu'on exécute le code ci-dessous, ce dernier générera les 2 fichiers (.csv et .xlsx) de manière automatique et cyclique. Pour ce notebook, j'ai choisi une exécution toutes les minutes, mais possibilité de choisir tous les jours, semaines, etc...

In [ ]:
# ------------------------------ Import des Librairies

import pandas as pd
import requests
import sqlite3
import datetime
!pip install schedule
import schedule
import time

# ------------------------------ Définition de la fonction 'batch'

def batch():

  # ------------------------------ Import de la DataBase

  link = 'https://github.com/SASambath69/wcs/raw/master/chinook.db'
  r = requests.get(link)
  open('chinook.db', 'wb').write(r.content)
  conn = sqlite3.connect('chinook.db')
  cursor = conn.cursor()

  # ------------------------------ Sauvegarde des tables dans des DataFrames

  media_types = pd.read_sql("select * from media_types", conn)
  genres = pd.read_sql("select * from genres", conn)
  playlists = pd.read_sql("select * from playlists", conn)
  playlist_track = pd.read_sql("select * from playlist_track", conn)
  tracks = pd.read_sql("select * from tracks", conn)
  artists = pd.read_sql("select * from artists", conn)
  invoices = pd.read_sql("select * from invoices", conn)
  invoice_items = pd.read_sql("select * from invoice_items", conn)
  albums = pd.read_sql("select * from albums", conn)
  customers = pd.read_sql("select * from customers", conn)
  employees = pd.read_sql("select * from employees", conn)

  # ------------------------------ Merge des tables "musicales"

  merge_1 = pd.merge(tracks, media_types.rename(columns={'Name':'MediaType'}), how='left', on='MediaTypeId').drop(columns='MediaTypeId')
  merge_2 = pd.merge(merge_1, genres.rename(columns={'Name':'Genre'}), how='left', on='GenreId').drop(columns='GenreId')
  merge_3 = pd.merge(playlist_track, playlists.rename(columns={'Name':'Playlist'}), how='left', on='PlaylistId').drop(columns='PlaylistId')
  merge_3_dummies = pd.get_dummies(merge_3).pivot_table(index='TrackId', aggfunc='sum').reset_index()

  def reset_dummies(x):
    if x==2:
      x=1
    return x

  merge_3_dummies['Playlist_Music'] = merge_3_dummies['Playlist_Music'].apply(reset_dummies)
  merge_4 = pd.merge(merge_2, merge_3_dummies, how='left', on='TrackId')
  merge_5 = pd.merge(albums.rename(columns={'Title':'Album'}), artists.rename(columns={'Name':'Artist'}), how='left', on='ArtistId').drop(columns='ArtistId')
  df_tracks = pd.merge(merge_4, merge_5, how='left', on='AlbumId').drop(columns=['AlbumId','UnitPrice'])

  # ------------------------------ Merge des tables "commerciales"

  customers_col = []
  for i in range(len(customers.columns)):
    customers_col.append('Customer' + customers.columns[i])
  customers_col[0] = 'CustomerId'
  customers.columns = customers_col

  employees_col = []
  for i in range(len(employees.columns)):
    employees_col.append('Employee' + employees.columns[i])
  employees_col[0] = 'EmployeeId'
  employees.columns = employees_col

  merge_6 = pd.merge(customers, employees, left_on='CustomerSupportRepId', right_on='EmployeeId').drop(columns=['CustomerSupportRepId','EmployeeId'])
  merge_7 = pd.merge(invoices, merge_6, how='left', on='CustomerId')
  merge_7['CustomerId'] = merge_7['CustomerId'].astype('object') # Sinon n'apparait pas dans mon "describe" plus bas
  df_cial = pd.merge(invoice_items, merge_7, how='left', on='InvoiceId')
  df_cial['InvoiceDate'] = df_cial['InvoiceDate'].apply(lambda x: x[:10])
  df_cial['EmployeeBirthDate'] = df_cial['EmployeeBirthDate'].apply(lambda x: x[:10])
  df_cial['EmployeeHireDate'] = df_cial['EmployeeHireDate'].apply(lambda x: x[:10])
  df_cial = df_cial.drop(columns=['CustomerId','BillingAddress','BillingCity','BillingState','BillingPostalCode','Total','CustomerState','EmployeeTitle','EmployeeReportsTo','EmployeeAddress','EmployeeCity','EmployeeState','EmployeeCountry','EmployeePostalCode'])
  df_cial['CustomerIdentity'] = df_cial['CustomerLastName'] + ' ' + df_cial['CustomerFirstName']
  df_cial['EmployeeIdentity'] = df_cial['EmployeeLastName'] + ' ' + df_cial['EmployeeFirstName']
  df_cial['TotalPrice'] = df_cial['UnitPrice'] * df_cial['Quantity']
  df_cial['EmployeeSeniority'] = df_cial['EmployeeHireDate'].apply(lambda x: round(int((datetime.date.today() - datetime.date(int(x[:4]),int(x[5:7]),int(x[8:]))).days) / 365,2))

  # ------------------------------ Merge de toutes les tables

  df_all = pd.merge(df_cial, df_tracks, how='left', on='TrackId')

  # ------------------------------ Enregistrement du DataFrame dans un fichier "csv" pour traitement ultérieur sur Tableau

  df_all.to_csv('df_all.csv')

  # ------------------------------ Création d'un fichier de Reporting sur Excel

  with pd.ExcelWriter('reporting_{}.xlsx'.format(str(datetime.date.today()))) as writer:

    # ------------------------------ Résumé

    df_all.describe(include='all')[['InvoiceDate','BillingCountry','CustomerIdentity','EmployeeIdentity','Name','Composer','MediaType','Genre','Album','Artist']].iloc[:4,:].to_excel(writer, sheet_name='Résumé')

    # ------------------------------ Répartition Géographique du CA

    sales_1_geo = invoices.pivot_table(index='BillingCountry', values='Total', aggfunc='sum').reset_index().sort_values('Total', ascending = False)
    sales_1_geo['Part %'] = round(sales_1_geo['Total'] / sales_1_geo['Total'].sum() * 100,2)
    sales_1_geo.to_excel(writer, sheet_name='Répartition Géographique du CA')

    # ------------------------------ Evolution du CA par Date

    df_all.pivot_table(index='InvoiceDate', values='TotalPrice', aggfunc='sum').reset_index().to_excel(writer, sheet_name='Evolution du CA par Date')

    # ------------------------------ CA par Mois par Année

    sales_2_date = df_all.pivot_table(index='InvoiceDate', values='TotalPrice', aggfunc='sum').reset_index()
    sales_2_date['Year'] = sales_2_date['InvoiceDate'].apply(lambda x: x[:4])
    sales_2_date['Month'] = sales_2_date['InvoiceDate'].apply(lambda x: int(x[5:7]))
    sales_2_date = sales_2_date.pivot_table(values='TotalPrice', index='Month', columns='Year', aggfunc='sum').reset_index()
    months = ['Janvier','Févrer','Mars','Avril','Mai','Juin','Juillet','Août','Septembre','Octobre','Novembre','Décembre']
    sales_2_date['MonthLib'] = months
    sales_2_date.reindex(columns=['MonthLib','2009','2010','2011','2012','2013']).to_excel(writer, sheet_name='CA par Mois par Année')

    # ------------------------------ CA par Vendeur

    df_all.pivot_table(index='EmployeeIdentity', values='TotalPrice', aggfunc='sum').reset_index().sort_values('TotalPrice', ascending=False).to_excel(writer, sheet_name='CA par Vendeur')

    # ------------------------------ Top 10 Clients

    df_all.pivot_table(index='CustomerIdentity', values='TotalPrice', aggfunc='sum').reset_index().sort_values('TotalPrice', ascending=False).head(10).to_excel(writer, sheet_name='Top 10 Clients')

    # ------------------------------ Top 10 Pistes

    pd.DataFrame({'Name':df_all['Name'].value_counts().head(10).index,'Quantity':df_all['Name'].value_counts().head(10).values}).to_excel(writer, sheet_name='Top 10 Pistes')

    # ------------------------------ Top 10 Albums

    pd.DataFrame({'Album':df_all['Album'].value_counts().head(10).index,'Quantity':df_all['Album'].value_counts().head(10).values}).to_excel(writer, sheet_name='Top 10 Albums')

    # ------------------------------ Top 10 Artistes

    pd.DataFrame({'Artist':df_all['Artist'].value_counts().head(10).index,'Quantity':df_all['Artist'].value_counts().head(10).values}).to_excel(writer, sheet_name='Top 10 Artistes')

    # ------------------------------ Préférences Genres

    pref_genres = df_all.pivot_table(index='Genre', values='Quantity', aggfunc='sum').reset_index().sort_values('Quantity', ascending=False)
    pref_genres['Part %'] = round(pref_genres['Quantity'] / pref_genres['Quantity'].sum() * 100, 2)
    pref_genres.to_excel(writer, sheet_name='Préférences Genres')

    # ------------------------------ Préférences Format

    pref_format = df_all.pivot_table(index='MediaType', values='Quantity', aggfunc='sum').reset_index().sort_values('Quantity', ascending=False)
    pref_format['Part %'] = round(pref_format['Quantity'] / pref_format['Quantity'].sum() * 100, 2)
    pref_format.to_excel(writer, sheet_name='Préférences Format')

    # ------------------------------ Infos Employés

    employees['EmployeeBirthDate'] = employees['EmployeeBirthDate'].apply(lambda x: x[:10])
    employees['EmployeeHireDate'] = employees['EmployeeHireDate'].apply(lambda x: x[:10])
    employees['EmployeeSeniority'] = employees['EmployeeHireDate'].apply(lambda x: round(int((datetime.date.today() - datetime.date(int(x[:4]),int(x[5:7]),int(x[8:]))).days) / 365,2))
    employees['EmployeeAge'] = employees['EmployeeBirthDate'].apply(lambda x: round(int((datetime.date.today() - datetime.date(int(x[:4]),int(x[5:7]),int(x[8:]))).days) / 365),0)
    employees['EmployeeIdentity'] = employees['EmployeeLastName'] + ' ' + employees['EmployeeFirstName']
    employees[['EmployeeIdentity','EmployeeBirthDate','EmployeeAge','EmployeeHireDate','EmployeeSeniority']].to_excel(writer, sheet_name='Infos Employés')

  now = datetime.datetime.now()
  dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
  print("Report Done at {}".format(dt_string))
  conn.close()

# ------------------------------ Définition de la périodicité du batch

schedule.clear()
schedule.every(1).minutes.do(batch)

# ------------------------------ Run or sleep

while True:
    schedule.run_pending()
    time.sleep(1)

# DataViz

Ci-dessous des screenshots des visualisations sur Tableau basées sur le fichier .csv généré par le code précédent. Vous pouvez les retrouver via les liens suivants :

*    Dashboard sur les ventes : [Lien](https://public.tableau.com/profile/sous8489#!/vizhome/Checkpoint4Sales/Sales)

![Dashboard sur les ventes](https://drive.google.com/uc?id=1ykEnnACLDGMw86JZP_mELjkF3JrjbZ3U)

*    Dashboard sur les préférences : [Lien](https://public.tableau.com/profile/sous8489#!/vizhome/Checkpoint4Prfrences/Prfrences)

![Dashboard sur les préférences](https://drive.google.com/uc?id=15AFyZoHh53aXR24swFj7rjoPcyiY02G3)

# 2. Système de Recommandations de Musiques

## Préparation du DataFrame qui servira de base au système de recommandations

Code à exécuter avant de passer à la suite

In [2]:
# ------------------------------ Import des Librairies

import pandas as pd
import requests
import sqlite3
import datetime
from sklearn.neighbors import KNeighborsClassifier

# ------------------------------ Import de la DataBase

link = 'https://github.com/SASambath69/wcs/raw/master/chinook.db'
r = requests.get(link)
open('chinook.db', 'wb').write(r.content)
conn = sqlite3.connect('chinook.db')
cursor = conn.cursor()

# ------------------------------ Sauvegarde des tables dans des DataFrames

media_types = pd.read_sql("select * from media_types", conn)
genres = pd.read_sql("select * from genres", conn)
playlists = pd.read_sql("select * from playlists", conn)
playlist_track = pd.read_sql("select * from playlist_track", conn)
tracks = pd.read_sql("select * from tracks", conn)
artists = pd.read_sql("select * from artists", conn)
invoices = pd.read_sql("select * from invoices", conn)
invoice_items = pd.read_sql("select * from invoice_items", conn)
albums = pd.read_sql("select * from albums", conn)
customers = pd.read_sql("select * from customers", conn)
employees = pd.read_sql("select * from employees", conn)

# ------------------------------ Merge des tables "musicales"

merge_1 = pd.merge(tracks, media_types.rename(columns={'Name':'MediaType'}), how='left', on='MediaTypeId').drop(columns='MediaTypeId')
merge_2 = pd.merge(merge_1, genres.rename(columns={'Name':'Genre'}), how='left', on='GenreId').drop(columns='GenreId')
merge_3 = pd.merge(playlist_track, playlists.rename(columns={'Name':'Playlist'}), how='left', on='PlaylistId').drop(columns='PlaylistId')
merge_3_dummies = pd.get_dummies(merge_3).pivot_table(index='TrackId', aggfunc='sum').reset_index()

def reset_dummies(x):
  if x==2:
    x=1
  return x

merge_3_dummies['Playlist_Music'] = merge_3_dummies['Playlist_Music'].apply(reset_dummies)
merge_4 = pd.merge(merge_2, merge_3_dummies, how='left', on='TrackId')
merge_5 = pd.merge(albums.rename(columns={'Title':'Album'}), artists.rename(columns={'Name':'Artist'}), how='left', on='ArtistId').drop(columns='ArtistId')
df_tracks = pd.merge(merge_4, merge_5, how='left', on='AlbumId').drop(columns=['AlbumId','UnitPrice'])

# ------------------------------ Merge des tables "commerciales"

customers_col = []
for i in range(len(customers.columns)):
  customers_col.append('Customer' + customers.columns[i])
customers_col[0] = 'CustomerId'
customers.columns = customers_col

employees_col = []
for i in range(len(employees.columns)):
  employees_col.append('Employee' + employees.columns[i])
employees_col[0] = 'EmployeeId'
employees.columns = employees_col

merge_6 = pd.merge(customers, employees, left_on='CustomerSupportRepId', right_on='EmployeeId').drop(columns=['CustomerSupportRepId','EmployeeId'])
merge_7 = pd.merge(invoices, merge_6, how='left', on='CustomerId')
merge_7['CustomerId'] = merge_7['CustomerId'].astype('object') # Sinon n'apparait pas dans mon "describe" plus bas
df_cial = pd.merge(invoice_items, merge_7, how='left', on='InvoiceId')
df_cial['InvoiceDate'] = df_cial['InvoiceDate'].apply(lambda x: x[:10])
df_cial['EmployeeBirthDate'] = df_cial['EmployeeBirthDate'].apply(lambda x: x[:10])
df_cial['EmployeeHireDate'] = df_cial['EmployeeHireDate'].apply(lambda x: x[:10])
df_cial = df_cial.drop(columns=['CustomerId','BillingAddress','BillingCity','BillingState','BillingPostalCode','Total','CustomerState','EmployeeTitle','EmployeeReportsTo','EmployeeAddress','EmployeeCity','EmployeeState','EmployeeCountry','EmployeePostalCode'])
df_cial['CustomerIdentity'] = df_cial['CustomerLastName'] + ' ' + df_cial['CustomerFirstName']
df_cial['EmployeeIdentity'] = df_cial['EmployeeLastName'] + ' ' + df_cial['EmployeeFirstName']
df_cial['TotalPrice'] = df_cial['UnitPrice'] * df_cial['Quantity']
df_cial['EmployeeSeniority'] = df_cial['EmployeeHireDate'].apply(lambda x: round(int((datetime.date.today() - datetime.date(int(x[:4]),int(x[5:7]),int(x[8:]))).days) / 365,2))

# ------------------------------ Merge de toutes les tables

df_all = pd.merge(df_cial, df_tracks, how='left', on='TrackId')

# ------------------------------ Get_dummies sur les Composers

split_composer = pd.concat([df_tracks[['TrackId','Composer']], df_tracks['Composer'].str.split(r", |&|/",expand=True)], axis=1)
melt_composer = split_composer.melt(id_vars=['TrackId','Composer'], value_name='Composers')
melt_composer_no_na = melt_composer.dropna()[['TrackId','Composers']]
melt_composer_no_na_unique = melt_composer_no_na.drop_duplicates(subset=['TrackId','Composers']).sort_values('TrackId').rename(columns={'Composers':'Composer'})
dummies_composer = pd.get_dummies(melt_composer_no_na_unique).pivot_table(index='TrackId', aggfunc='sum').reset_index()

# ------------------------------ Get_dummies sur les MediaTypes

dummies_mediatype = pd.get_dummies(df_tracks[['TrackId','MediaType']])

# ------------------------------ Get_dummies sur les Genres

dummies_genre = pd.get_dummies(df_tracks[['TrackId','Genre']])

# ------------------------------ Get_dummies sur les Albums

dummies_album = pd.get_dummies(df_tracks[['TrackId','Album']])

# ------------------------------ Get_dummies sur les Artists

split_artist = pd.concat([df_tracks[['TrackId','Artist']], df_tracks['Artist'].str.split(r", |&",expand=True)], axis=1)
melt_artist = split_artist.melt(id_vars=['TrackId','Artist'], value_name='Artists')
melt_artist_no_na = melt_artist.dropna()[['TrackId','Artists']]
melt_artist_no_na_unique = melt_artist_no_na.drop_duplicates(subset=['TrackId','Artists']).sort_values('TrackId').rename(columns={'Artists':'Artist'})
dummies_artist = pd.get_dummies(melt_artist_no_na_unique).pivot_table(index='TrackId', aggfunc='sum').reset_index()

# ------------------------------ Merge

df_tracks_ml = df_tracks.drop(columns=['Composer','MediaType','Genre','Album','Artist'])
merge_8 = pd.merge(df_tracks_ml, dummies_composer, how='left', on='TrackId')
merge_9 = pd.merge(merge_8, dummies_mediatype, how='left', on='TrackId')
merge_10 = pd.merge(merge_9, dummies_genre, how='left', on='TrackId')
merge_11 = pd.merge(merge_10, dummies_album, how='left', on='TrackId')
merge_12 = pd.merge(merge_11, dummies_artist, how='left', on='TrackId')

# ------------------------------ Préparation des différents DataFrames sources

# DataFrame des "Composers"
df_composer = pd.concat([merge_12.iloc[:,:2],merge_12.iloc[:,16:1156]], axis=1).loc[pd.concat([merge_12.iloc[:,:2],merge_12.iloc[:,16:1156]], axis=1).isna().any(axis=1)==False].reset_index(drop=True)

# DataFrame des "Genres"
df_genre = pd.concat([merge_12.iloc[:,:2],merge_12.iloc[:,1151:1181]], axis=1).reset_index(drop=True)

# DataFrame des "Artists" et "Albums"
df_artist = pd.concat([merge_12.iloc[:,:2], merge_12.iloc[:,1151:1156], merge_12.iloc[:,1181:]], axis=1).reset_index(drop=True)

## Exécution du système

Afin de vérifier le bon fonctionnement du système :

*    Au moment de l'exécution du 1er bloc de code, sélectionner un numéro entre 1 et 3 503. Ces chiffres correspondent aux index des musiques. Je laisse un choix humain pour cela, plutôt que de partir sur un "vrai random"
*    A l'exécution du 2ème bloc, copier-coller le nom de la musique en input. Le code proposera alors plusieurs sélections en fonction du compositeur, du genre, puis des artistes & albums.

In [ ]:
df_tracks[['TrackId','Name']].loc[df_tracks['TrackId'] == int(input('Please select an Id between 1 and 3503 : '))]

Please select an Id between 1 and 3503 : 12


,TrackId,Name
11,12,Breaking The Rules


In [4]:
# Pour pouvoir afficher tous les outputs et non pas seulement le dernier

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# ------------------------------ Recommandations en fonction des "Composers"

saisie = input("Please select a track : ")
X = df_composer.iloc[:,2:]
y = df_composer['Name']==saisie

if (df_composer['Name']==saisie).sum() == 0:
  
  print('Sorry, not possible !')

else:
  
  # Trouver les plus proches voisins

  model_knc = KNeighborsClassifier(n_neighbors=6).fit(X, y)
  neighbors = model_knc.kneighbors(X[y])

  close = neighbors[1][0,1:]

  print()
  print("Recommandations en fonction des Composers : ")
  print()

  recommanded = [saisie]
  for i in close:
    print("- ", df_composer['Name'][i])
    recommanded.append(df_composer['Name'][i])

  print()
pd.concat([df_tracks.iloc[:,:7], df_tracks.iloc[:,-2:]], axis=1)[pd.concat([df_tracks.iloc[:,:7], df_tracks.iloc[:,-2:]], axis=1)['Name'].isin(recommanded)]

# ------------------------------ Recommandations en fonction des "Genres"

X = df_genre.iloc[:,2:]
y = df_genre['Name']==saisie

# Trouver les plus proches voisins

model_knc = KNeighborsClassifier(n_neighbors=6).fit(X, y)
neighbors = model_knc.kneighbors(X[y])

close = neighbors[1][0,1:]

print()
print("Recommandations en fonction des Genres : ")
print()

recommanded = [saisie]
for i in close:
  print("- ", df_genre['Name'][i])
  recommanded.append(df_genre['Name'][i])

print()
pd.concat([df_tracks.iloc[:,:7], df_tracks.iloc[:,-2:]], axis=1)[pd.concat([df_tracks.iloc[:,:7], df_tracks.iloc[:,-2:]], axis=1)['Name'].isin(recommanded)]

# ------------------------------ Recommandations en fonction des "Artists/Albums"

X = df_artist.iloc[:,2:]
y = df_artist['Name']==saisie

# Trouver les plus proches voisins

model_knc = KNeighborsClassifier(n_neighbors=6).fit(X, y)
neighbors = model_knc.kneighbors(X[y])

close = neighbors[1][0,1:]

print()
print("Recommandations en fonction des Artists & Albums : ")
print()

recommanded = [saisie]
for i in close:
  print("- ", df_artist['Name'][i])
  recommanded.append(df_artist['Name'][i])

print()
pd.concat([df_tracks.iloc[:,:7], df_tracks.iloc[:,-2:]], axis=1)[pd.concat([df_tracks.iloc[:,:7], df_tracks.iloc[:,-2:]], axis=1)['Name'].isin(recommanded)]

Please select a track : Breaking The Rules

Recommandations en fonction des Composers : 

-  Evil Walks
-  Inject The Venom
-  Put The Finger On You
-  Let's Get It Up
-  Snowballed



,TrackId,Name,Composer,Milliseconds,Bytes,MediaType,Genre,Album,Artist
5,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
6,7,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
7,8,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
8,9,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
9,10,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
11,12,Breaking The Rules,"Angus Young, Malcolm Young, Brian Johnson",263288,8596840,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC



Recommandations en fonction des Genres : 

-  It Ain't Like That
-  Love, Hate, Love
-  I Can't Remember
-  The Show Must Go On
-  Bleed The Freak



,TrackId,Name,Composer,Milliseconds,Bytes,MediaType,Genre,Album,Artist
11,12,Breaking The Rules,"Angus Young, Malcolm Young, Brian Johnson",263288,8596840,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
53,54,Bleed The Freak,Jerry Cantrell,241946,7847716,MPEG audio file,Rock,Facelift,Alice In Chains
54,55,I Can't Remember,"Jerry Cantrell, Layne Staley",222955,7302550,MPEG audio file,Rock,Facelift,Alice In Chains
55,56,"Love, Hate, Love","Jerry Cantrell, Layne Staley",387134,12575396,MPEG audio file,Rock,Facelift,Alice In Chains
56,57,It Ain't Like That,"Jerry Cantrell, Michael Starr, Sean Kinney",277577,8993793,MPEG audio file,Rock,Facelift,Alice In Chains
433,434,The Show Must Go On,Queen,263784,8526760,MPEG audio file,Rock,Greatest Hits II,Queen



Recommandations en fonction des Artists & Albums : 

-  Evil Walks
-  Let's Get It Up
-  For Those About To Rock (We Salute You)
-  Put The Finger On You
-  Snowballed



,TrackId,Name,Composer,Milliseconds,Bytes,MediaType,Genre,Album,Artist
0,1,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
5,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
6,7,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
8,9,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
9,10,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
11,12,Breaking The Rules,"Angus Young, Malcolm Young, Brian Johnson",263288,8596840,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
